In [7]:
# Лабораторная работа
## Ансамбли моделей машинного обучения. Часть 2.

#Цель работы**: изучение ансамблей моделей машинного обучения.

#Используемый датасет**: Processed_Plant_Growth_Metrics.csv


In [8]:
# Установка необходимых библиотек
#pip install scikit-learn pandas numpy matplotlib seaborn gmdh


In [9]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import gmdh
from gmdh.gmdh import GMDH_COMBI as COMBI, GMDH_MULTI as MULTI  # возможные альтернативные имена

# Настройка отображения графиков
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set(font_scale=1.2)

# Для воспроизводимости результатов
np.random.seed(42)


ImportError: cannot import name 'GMDH_COMBI' from 'gmdh.gmdh' (c:\Users\bekab\AppData\Local\Programs\Python\Python39\lib\site-packages\gmdh\gmdh.py)

In [ ]:
# Загрузка данных
df = pd.read_csv('Processed_Plant_Growth_Metrics.csv')

# Вывод первых 5 строк данных
print("Первые 5 строк набора данных:")
display(df.head())

# Информация о наборе данных
print("\nИнформация о наборе данных:")
display(df.info())

# Статистическое описание
print("\nСтатистическое описание набора данных:")
display(df.describe())


In [ ]:
# Проверка на пропущенные значения
print("Количество пропущенных значений в каждом столбце:")
display(df.isnull().sum())

# Проверка наличия дубликатов
print(f"\nКоличество дубликатов: {df.duplicated().sum()}")

# Анализ целевой переменной
print("\nАнализ целевой переменной (Growth_Rate):")
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.histplot(df['Growth_Rate'], kde=True)
plt.title('Распределение Growth_Rate')

plt.subplot(1, 2, 2)
sns.boxplot(y=df['Growth_Rate'])
plt.title('Boxplot для Growth_Rate')
plt.tight_layout()
plt.show()

# Корреляционная матрица
plt.figure(figsize=(14, 10))
corr = df.select_dtypes(include=[np.number]).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', center=0)
plt.title('Корреляционная матрица числовых признаков')
plt.tight_layout()
plt.show()


In [ ]:
# Подготовка данных для моделирования

# Определение числовых и категориальных признаков
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Удаляем целевую переменную из списка числовых признаков
if 'Growth_Rate' in numeric_features:
    numeric_features.remove('Growth_Rate')

print("Числовые признаки:", numeric_features)
print("Категориальные признаки:", categorical_features)

# Разделение на признаки и целевую переменную
X = df.drop('Growth_Rate', axis=1)
y = df['Growth_Rate']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

# Создание препроцессора для числовых и категориальных признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединение преобразователей
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Применение препроцессора к данным
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

print(f"Размер обучающей выборки после препроцессинга: {X_train_preprocessed.shape}")
print(f"Размер тестовой выборки после препроцессинга: {X_test_preprocessed.shape}")


In [ ]:
# Модель 1: Стекинг (Stacking)
from sklearn.ensemble import StackingRegressor

# Определение базовых моделей для стекинга
base_models = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('gbr', GradientBoostingRegressor(random_state=42)),
    ('ridge', Ridge(random_state=42))
]

# Создание модели стекинга с линейной регрессией в качестве мета-модели
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=LinearRegression(),
    cv=5
)

# Обучение стекинг-модели
print("Обучение стекинг-модели...")
stacking_model.fit(X_train_preprocessed, y_train)

# Предсказание значений
y_pred_stacking = stacking_model.predict(X_test_preprocessed)

# Оценка качества модели
mse_stacking = mean_squared_error(y_test, y_pred_stacking)
rmse_stacking = np.sqrt(mse_stacking)
mae_stacking = mean_absolute_error(y_test, y_pred_stacking)
r2_stacking = r2_score(y_test, y_pred_stacking)

print("Метрики качества для стекинг-модели:")
print(f"MSE: {mse_stacking:.4f}")
print(f"RMSE: {rmse_stacking:.4f}")
print(f"MAE: {mae_stacking:.4f}")
print(f"R²: {r2_stacking:.4f}")

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_stacking, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Фактические значения')
plt.ylabel('Предсказанные значения')
plt.title('Стекинг-модель: предсказанные vs фактические значения')
plt.grid(True)
plt.show()


In [ ]:
# Модель 2: Многослойный персептрон (МLP)
# Создание и обучение MLP модели
mlp_model = MLPRegressor(
    hidden_layer_sizes=(100, 50, 25), 
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='adaptive',
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    verbose=True
)

print("Обучение модели многослойного персептрона...")
mlp_model.fit(X_train_preprocessed, y_train)

# Предсказание значений
y_pred_mlp = mlp_model.predict(X_test_preprocessed)

# Оценка качества модели
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
rmse_mlp = np.sqrt(mse_mlp)
mae_mlp = mean_absolute_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print("Метрики качества для модели многослойного персептрона:")
print(f"MSE: {mse_mlp:.4f}")
print(f"RMSE: {rmse_mlp:.4f}")
print(f"MAE: {mae_mlp:.4f}")
print(f"R²: {r2_mlp:.4f}")

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_mlp, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Фактические значения')
plt.ylabel('Предсказанные значения')
plt.title('Многослойный персептрон: предсказанные vs фактические значения')
plt.grid(True)
plt.show()


In [ ]:
# Подготовка данных для моделей МГУА
# Для моделей МГУА нам нужны numpy массивы
X_train_np = X_train_preprocessed.copy()
X_test_np = X_test_preprocessed.copy()

# Проверка, является ли X_train_preprocessed разреженной матрицей
from scipy.sparse import issparse

if issparse(X_train_preprocessed):
    X_train_np = X_train_preprocessed.toarray()
    X_test_np = X_test_preprocessed.toarray()

print(f"Форма данных для МГУА моделей - X_train: {X_train_np.shape}, X_test: {X_test_np.shape}")

# Модель 3: COMBI (линейный метод МГУА)
print("Обучение модели COMBI...")
combi_model = COMBI(
    ref_functions=('const', 'linear'),  # Используем константу и линейную функцию в качестве опорных
    criterion_type='test_mse',          # Критерий качества - MSE на тестовой выборке
    criterion_minimize=True,             # Минимизировать критерий
    p_max=3,                             # Максимальное число переменных в модели
    verbose=True                         # Вывод информации о ходе обучения
)

# Разделение данных на обучающую и валидационную выборки для МГУА
from sklearn.model_selection import train_test_split
X_train_gmdh, X_val_gmdh, y_train_gmdh, y_val_gmdh = train_test_split(
    X_train_np, y_train, test_size=0.25, random_state=42
)

# Обучение модели COMBI
combi_model.fit(X_train_gmdh, y_train_gmdh, X_val_gmdh, y_val_gmdh)

# Предсказание на тестовой выборке
y_pred_combi = combi_model.predict(X_test_np)

# Оценка качества модели
mse_combi = mean_squared_error(y_test, y_pred_combi)
rmse_combi = np.sqrt(mse_combi)
mae_combi = mean_absolute_error(y_test, y_pred_combi)
r2_combi = r2_score(y_test, y_pred_combi)

print("Метрики качества для модели COMBI:")
print(f"MSE: {mse_combi:.4f}")
print(f"RMSE: {rmse_combi:.4f}")
print(f"MAE: {mae_combi:.4f}")
print(f"R²: {r2_combi:.4f}")

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_combi, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Фактические значения')
plt.ylabel('Предсказанные значения')
plt.title('Модель COMBI: предсказанные vs фактические значения')
plt.grid(True)
plt.show()


In [ ]:
# Модель 4: MIA (нелинейный метод МГУА)
print("Обучение модели MIA...")
mia_model = MIA(
    ref_functions=('const', 'linear', 'quad'),  # Используем константу, линейную и квадратичную функции
    max_layer_count=5,              # Максимальное число слоев
    criterion_type='test_mse',      # Критерий качества - MSE на тестовой выборке
    criterion_minimize=True,         # Минимизировать критерий
    p_max=3,                         # Максимальное число переменных в модели
    verbose=True                     # Вывод информации о ходе обучения
)

# Обучение модели MIA
mia_model.fit(X_train_gmdh, y_train_gmdh, X_val_gmdh, y_val_gmdh)

# Предсказание на тестовой выборке
y_pred_mia = mia_model.predict(X_test_np)

# Оценка качества модели
mse_mia = mean_squared_error(y_test, y_pred_mia)
rmse_mia = np.sqrt(mse_mia)
mae_mia = mean_absolute_error(y_test, y_pred_mia)
r2_mia = r2_score(y_test, y_pred_mia)

print("Метрики качества для модели MIA:")
print(f"MSE: {mse_mia:.4f}")
print(f"RMSE: {rmse_mia:.4f}")
print(f"MAE: {mae_mia:.4f}")
print(f"R²: {r2_mia:.4f}")

# Визуализация результатов
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_mia, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Фактические значения')
plt.ylabel('Предсказанные значения')
plt.title('Модель MIA: предсказанные vs фактические значения')
plt.grid(True)
plt.show()


In [ ]:
# Сравнение всех моделей
models = {
    'Stacking': [mse_stacking, rmse_stacking, mae_stacking, r2_stacking],
    'MLP': [mse_mlp, rmse_mlp, mae_mlp, r2_mlp],
    'COMBI (МГУА)': [mse_combi, rmse_combi, mae_combi, r2_combi],
    'MIA (МГУА)': [mse_mia, rmse_mia, mae_mia, r2_mia]
}

# Создание таблицы сравнения
comparison_df = pd.DataFrame.from_dict(models, 
                                       orient='index',
                                       columns=['MSE', 'RMSE', 'MAE', 'R²'])
display(comparison_df)

# Визуализация сравнения по метрикам
plt.figure(figsize=(14, 5))

# RMSE
plt.subplot(1, 3, 1)
comparison_df['RMSE'].plot(kind='bar', color='skyblue')
plt.title('RMSE для всех моделей')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.grid(axis='y')

# MAE
plt.subplot(1, 3, 2)
comparison_df['MAE'].plot(kind='bar', color='lightgreen')
plt.title('MAE для всех моделей')
plt.ylabel('MAE')
plt.xticks(rotation=45)
plt.grid(axis='y')

# R²
plt.subplot(1, 3, 3)
comparison_df['R²'].plot(kind='bar', color='salmon')
plt.title('R² для всех моделей')
plt.ylabel('R²')
plt.xticks(rotation=45)
plt.grid(axis='y')

plt.tight_layout()
plt.show()

# Визуализация предсказаний лучшей модели (определяем по наилучшему R²)
best_model = comparison_df['R²'].idxmax()
print(f"Лучшая модель по метрике R²: {best_model}")

# Получаем предсказания лучшей модели
if best_model == 'Stacking':
    y_pred_best = y_pred_stacking
elif best_model == 'MLP':
    y_pred_best = y_pred_mlp
elif best_model == 'COMBI (МГУА)':
    y_pred_best = y_pred_combi
elif best_model == 'MIA (МГУА)':
    y_pred_best = y_pred_mia

# Визуализация предсказаний лучшей модели
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_best, alpha=0.5, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Фактические значения')
plt.ylabel('Предсказанные значения')
plt.title(f'Лучшая модель ({best_model}): предсказанные vs фактические значения')
plt.grid(True)
plt.show()


In [ ]:
# Оценка важности признаков для лучшей модели
# Для стекинга можно оценить важность признаков по базовым моделям
# Например, если RandomForestRegressor был лучшим из базовых моделей

if best_model == 'Stacking':
    # Берем базовую модель RandomForest из стекинга
    for name, model in stacking_model.estimators_:
        if name == 'rf':
            rf_model = model
            
    # Если есть преобразователь признаков для стекинга
    feature_names = []
    if hasattr(preprocessor, 'get_feature_names_out'):
        feature_names = preprocessor.get_feature_names_out()
    else:
        feature_names = [f'feature_{i}' for i in range(X_train_preprocessed.shape[1])]
    
    # Получаем важность признаков
    importances = rf_model.feature_importances_
    
    # Создаем DataFrame для сортировки
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })
    
    # Сортировка по важности
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
    
    # Визуализация
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
    plt.title('Топ-20 важных признаков (RandomForest из стекинга)')
    plt.tight_layout()
    plt.show()

# Для MLP нет встроенного метода оценки важности признаков, но можно использовать 
# пермутационную важность
elif best_model == 'MLP':
    from sklearn.inspection import permutation_importance
    
    # Вычисление пермутационной важности
    perm_importance = permutation_importance(mlp_model, X_test_preprocessed, y_test, n_repeats=10, random_state=42)
    
    # Получаем имена признаков
    feature_names = []
    if hasattr(preprocessor, 'get_feature_names_out'):
        feature_names = preprocessor.get_feature_names_out()
    else:
        feature_names = [f'feature_{i}' for i in range(X_train_preprocessed.shape[1])]
    
    # Создаем DataFrame для сортировки
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': perm_importance.importances_mean
    })
    
    # Сортировка по важности
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
    
    # Визуализация
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
    plt.title('Топ-20 важных признаков (MLP - пермутационная важность)')
    plt.tight_layout()
    plt.show()
    
# Для моделей МГУА можно вывести структуру модели и коэффициенты
elif 'МГУА' in best_model:
    if best_model == 'COMBI (МГУА)':
        gmdh_model = combi_model
    else:
        gmdh_model = mia_model
        
    print(f"Информация о модели {best_model}:")
    print(f"Структура модели: {gmdh_model}")
    # Пытаемся вывести коэффициенты или другую информацию о модели
    if hasattr(gmdh_model, 'coefs_'):
        print("Коэффициенты модели:")
        for i, coef in enumerate(gmdh_model.coefs_):
            print(f"Коэффициент {i}: {coef}")
    
    # Попытка оценить важность признаков с помощью пермутационной важности
    from sklearn.inspection import permutation_importance
    
    # Получаем имена признаков
    feature_names = []
    if hasattr(preprocessor, 'get_feature_names_out'):
        feature_names = preprocessor.get_feature_names_out()
    else:
        feature_names = [f'feature_{i}' for i in range(X_train_np.shape[1])]
    
    # Вычисление пермутационной важности
    perm_importance = permutation_importance(gmdh_model, X_test_np, y_test, n_repeats=5, random_state=42)
    
    # Создаем DataFrame для сортировки
    feature_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': perm_importance.importances_mean
    })
    
    # Сортировка по важности
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)
    
    # Визуализация
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
    plt.title(f'Топ-20 важных признаков ({best_model} - пермутационная важность)')
    plt.tight_layout()
    plt.show()


In [ ]:
# Выводы по результатам работы

## Результаты и сравнение моделей:

1. **Стекинг (Stacking)**: Ансамблевый метод, объединяющий предсказания нескольких моделей (Random Forest, Gradient Boosting, Ridge) с помощью метамодели (линейная регрессия).

2. **Многослойный персептрон (MLP)**: Нейронная сеть с тремя скрытыми слоями (100, 50 и 25 нейронов).

3. **COMBI (линейный метод МГУА)**: Метод группового учета аргументов, использующий линейные зависимости.

4. **MIA (нелинейный метод МГУА)**: Метод группового учета аргументов с нелинейными опорными функциями.

По результатам сравнения метрик качества моделей, лучшие результаты показала [здесь будет название лучшей модели по R²]. Это демонстрирует эффективность [соответствующего подхода] для решения задачи регрессии на данном наборе данных.

## Особенности МГУА моделей:

МГУА модели показали [здесь будет результат сравнения МГУА с другими моделями]. Библиотека GMDH позволяет строить интерпретируемые модели с автоматическим выбором структуры, что является их преимуществом по сравнению с "черными ящиками" вроде нейронных сетей.

## Общий вывод:

В данной работе мы применили различные ансамблевые методы машинного обучения для решения задачи регрессии. Сравнение моделей показало, что для данного набора данных о росте растений наиболее эффективным методом является [здесь будет название лучшей модели].
